In [2]:
import os
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import box
import pandas as pd

# PATHS
mosaic_fp    = r"D:\1Research\2025\NOAA_SatHack\NBR\nbr_mosaic2_1km.tif"
county_shp   = r"D:\1Research\2025\NOAA_SatHack\cb_2018_us_county_500k\cb_2018_us_county_500k.shp"
output_excel = r"D:\1Research\2025\NOAA_SatHack\NBR\postfire_nbr.xlsx"

# LOAD and FILTER COUNTIES
counties    = gpd.read_file(county_shp)
ca_counties = counties[counties["STATEFP"] == "06"].copy()

# OPEN NBR MOSAIC and PREPARE BBOX
results = []
with rasterio.open(mosaic_fp) as src:
    # Reproject counties into the raster CRS
    ca_counties = ca_counties.to_crs(src.crs)
    # Build a bounding‐box polygon of the raster extent
    raster_bbox = box(*src.bounds)

    # Iterate over counties
    for _, row in ca_counties.iterrows():
        name    = row["NAME"]
        poly    = row.geometry

        # Skip if county polygon does not intersect the raster footprint
        if not poly.intersects(raster_bbox):
            mean_nbr = np.nan
        else:
            try:
                # Mask the raster by this county polygon
                out_image, _ = mask(
                    src,
                    [poly],
                    crop=True,
                    filled=True,
                    nodata=np.nan
                )
                data = out_image[0]  

                # Extract valid pixels and compute mean
                vals = data[~np.isnan(data)]
                mean_nbr = round(float(vals.sum() / vals.size), 2) if vals.size else np.nan

            except ValueError:
                # Catch any masking errors
                mean_nbr = np.nan

        results.append({
            "Name of County": name,
            "NBR value":      mean_nbr
        })

# SAVE TO EXCEL
df = pd.DataFrame(results).sort_values("Name of County").reset_index(drop=True)
df.to_excel(output_excel, index=False)
print(f"Saved county‐level NBR to: {output_excel}")


✅ Saved county‐level NBR to: D:\1Research\2025\NOAA_SatHack\NBR\postfire_nbr.xlsx
